<a href="https://colab.research.google.com/github/punjabinuclei/RealTimeBatteryMonitoringSystem/blob/main/Cycle%20Tester%20With%20Features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd # data processing
import numpy as np # working with arrays
from termcolor import colored as cl # text customization

from sklearn.model_selection import train_test_split # data split
from matplotlib import pyplot as plt


from sklearn import preprocessing # preprocessData
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score as r2_score # evaluation metric

In [2]:
# Models
# -------------------------------------------------------------

# 1.OLS----------
from sklearn.linear_model import LinearRegression # OLS algorithm


# 2. Decision Tree--------
from sklearn.tree import DecisionTreeRegressor

# 3. Random Forest-------
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

# 4.Gradient Boosting-------
from sklearn.ensemble import GradientBoostingRegressor

#5. Extreme Gradient Boosting--------
import xgboost as xgb


# 6 and 7 For NN and DNN-------------
import keras
from keras.models import Sequential
from keras.layers import Dense

# 8 Nd 9. SVM
from sklearn.svm import SVR


In [4]:
df = pd.read_csv("VAH01.csv")
df = df.drop(['time_s', 'EnergyCharge_W_h', 'QCharge_mA_h', 'EnergyDischarge_W_h', 'cycleNumber', 'Ns'], axis=1)
df

,Ecell_V,I_mA,QDischarge_mA_h,Temperature__C
0,3.538717,1491.3828,0.000000,21.139246
1,3.545570,1492.1708,0.000000,21.115524
2,3.549980,1491.5797,0.000000,21.305294
3,3.553131,1492.3679,0.000000,21.574135
4,3.555572,1490.9886,0.000000,21.510880
...,...,...,...,...
1048570,3.591331,-4464.8276,1088.086524,33.996197
1048571,3.586133,-4471.3296,1100.499700,33.774796
1048572,3.580974,-4477.8306,1112.930732,33.988289
1048573,3.575775,-4484.3325,1125.379697,33.964565


In [5]:
# ----------------------------------------------------------SOC column-----------------------------------------------------


df['SOC%'] = ((3000-df['QDischarge_mA_h'])/3000)*100



In [6]:
def sliding_window_mean(values, window_size):
    result = np.zeros(len(values) - window_size + 1)
    for i in range(len(result)):
        result[i] = np.mean(values[i:i+window_size])
    return result

# ---------------------------------------------------------------------
# Select the column of interest
column = df["Ecell_V"]

window_size = 10
mean_values = sliding_window_mean(column, window_size)

# Pad the mean_values array with NaNs to match the length of the DataFrame's index
padding = np.empty(len(df.index) - len(mean_values))
padding[:] = np.nan
mean_values = np.concatenate((mean_values, padding))

# Save the result to a new column in the DataFrame
df["EcellMean"] = mean_values

# ----------------------------------------------------------------------
# Select the column of interest
column = df["I_mA"]

window_size = 10
mean_values = sliding_window_mean(column, window_size)

# Pad the mean_values array with NaNs to match the length of the DataFrame's index
padding = np.empty(len(df.index) - len(mean_values))
padding[:] = np.nan
mean_values = np.concatenate((mean_values, padding))

# Save the result to a new column in the DataFrame
df["I_Mean"] = mean_values

# ----------------------------------------------------------------------
# Select the column of interest
column = df["Temperature__C"]

window_size = 10
mean_values = sliding_window_mean(column, window_size)

# Pad the mean_values array with NaNs to match the length of the DataFrame's index
padding = np.empty(len(df.index) - len(mean_values))
padding[:] = np.nan
mean_values = np.concatenate((mean_values, padding))

# Save the result to a new column in the DataFrame
df["TemperatureMean"] = mean_values

In [7]:
def sliding_window_median(values, window_size):
    result = np.zeros(len(values) - window_size + 1)
    for i in range(len(result)):
        result[i] = np.median(values[i:i+window_size])
    return result

# --------------------------------------------------------------


# Select the column of interest
column = df["Ecell_V"]

window_size = 10
median_values = sliding_window_median(column, window_size)

# Pad the median_values array with NaNs to match the length of the DataFrame's index
padding = np.empty(len(df.index) - len(median_values))
padding[:] = np.nan
median_values = np.concatenate((median_values, padding))

# Save the result to a new column in the DataFrame
df["EcellMedian"] = median_values



# -----------------------------------------------------------

# Select the column of interest
column = df["I_mA"]

window_size = 10
median_values = sliding_window_median(column, window_size)

# Pad the median_values array with NaNs to match the length of the DataFrame's index
padding = np.empty(len(df.index) - len(median_values))
padding[:] = np.nan
median_values = np.concatenate((median_values, padding))

# Save the result to a new column in the DataFrame
df["ImaMedian"] = median_values
df


# -------------------------------------------------------------------------------
# Select the column of interest
column = df["Temperature__C"]

window_size = 10
median_values = sliding_window_median(column, window_size)

# Pad the median_values array with NaNs to match the length of the DataFrame's index
padding = np.empty(len(df.index) - len(median_values))
padding[:] = np.nan
median_values = np.concatenate((median_values, padding))

# Save the result to a new column in the DataFrame
df["TempMedian"] = median_values
df

,Ecell_V,I_mA,QDischarge_mA_h,Temperature__C,SOC%,EcellMean,I_Mean,TemperatureMean,EcellMedian,ImaMedian,TempMedian
0,3.538717,1491.3828,0.000000,21.139246,100.000000,3.554757,1491.99352,21.653207,3.556596,1491.67830,21.657160
1,3.545570,1492.1708,0.000000,21.115524,100.000000,3.557431,1492.07233,21.777349,3.558487,1491.97385,21.791581
2,3.549980,1491.5797,0.000000,21.305294,100.000000,3.559566,1492.01323,21.896746,3.560180,1491.67835,21.922049
3,3.553131,1492.3679,0.000000,21.574135,100.000000,3.561413,1491.91473,22.016934,3.561775,1491.67835,22.068330
4,3.555572,1490.9886,0.000000,21.510880,100.000000,3.563087,1491.79652,22.117354,3.563311,1491.57975,22.151355
...,...,...,...,...,...,...,...,...,...,...,...
1048570,3.591331,-4464.8276,1088.086524,33.996197,63.730449,NaN,NaN,NaN,NaN,NaN,NaN
1048571,3.586133,-4471.3296,1100.499700,33.774796,63.316677,NaN,NaN,NaN,NaN,NaN,NaN
1048572,3.580974,-4477.8306,1112.930732,33.988289,62.902309,NaN,NaN,NaN,NaN,NaN,NaN
1048573,3.575775,-4484.3325,1125.379697,33.964565,62.487343,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# ------------------------------------------------------Standard Deviatoion------------------------------------

def sliding_window_stddev(values, window_size):
    result = np.zeros(len(values) - window_size + 1)
    for i in range(len(result)):
        result[i] = np.std(values[i:i+window_size])
    return result
# --------------------------------------------------

# Select the column of interest
column = df["Ecell_V"]

window_size = 10
stddev_values = sliding_window_stddev(column, window_size)

# Pad the stddev_values array with NaNs to match the length of the DataFrame's index
padding = np.empty(len(df.index) - len(stddev_values))
padding[:] = np.nan
stddev_values = np.concatenate((stddev_values, padding))

# Save the result to a new column in the DataFrame
df["Ecell_Vstd"] = stddev_values
df

# --------------------------------------------------------


# Select the column of interest
column = df["I_mA"]

window_size = 10
stddev_values = sliding_window_stddev(column, window_size)

# Pad the stddev_values array with NaNs to match the length of the DataFrame's index
padding = np.empty(len(df.index) - len(stddev_values))
padding[:] = np.nan
stddev_values = np.concatenate((stddev_values, padding))

# Save the result to a new column in the DataFrame
df["I_mA_std"] = stddev_values
df


# -----------------------------------------------------

# Select the column of interest
column = df["Temperature__C"]

window_size = 10
stddev_values = sliding_window_stddev(column, window_size)

# Pad the stddev_values array with NaNs to match the length of the DataFrame's index
padding = np.empty(len(df.index) - len(stddev_values))
padding[:] = np.nan
stddev_values = np.concatenate((stddev_values, padding))

# Save the result to a new column in the DataFrame
df["TempStd"] = stddev_values
df

,Ecell_V,I_mA,QDischarge_mA_h,Temperature__C,SOC%,EcellMean,I_Mean,TemperatureMean,EcellMedian,ImaMedian,TempMedian,Ecell_Vstd,I_mA_std,TempStd
0,3.538717,1491.3828,0.000000,21.139246,100.000000,3.554757,1491.99352,21.653207,3.556596,1491.67830,21.657160,0.007640,0.891832,0.369085
1,3.545570,1492.1708,0.000000,21.115524,100.000000,3.557431,1492.07233,21.777349,3.558487,1491.97385,21.791581,0.006078,0.868909,0.383817
2,3.549980,1491.5797,0.000000,21.305294,100.000000,3.559566,1492.01323,21.896746,3.560180,1491.67835,21.922049,0.005226,0.880226,0.342895
3,3.553131,1492.3679,0.000000,21.574135,100.000000,3.561413,1491.91473,22.016934,3.561775,1491.67835,22.068330,0.004754,0.973409,0.324673
4,3.555572,1490.9886,0.000000,21.510880,100.000000,3.563087,1491.79652,22.117354,3.563311,1491.57975,22.151355,0.004482,0.982928,0.327473
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,3.591331,-4464.8276,1088.086524,33.996197,63.730449,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1048571,3.586133,-4471.3296,1100.499700,33.774796,63.316677,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1048572,3.580974,-4477.8306,1112.930732,33.988289,62.902309,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1048573,3.575775,-4484.3325,1125.379697,33.964565,62.487343,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# -------------------------------------------------------------Variance-----------------------------------------------------


def sliding_window_variance(values, window_size):
    result = np.zeros(len(values) - window_size + 1)
    for i in range(len(result)):
        result[i] = np.var(values[i:i+window_size])
    return result


# -----------------------------------------------------------------

# Select the column of interest
column = df["Ecell_V"]

window_size = 10
variance_values = sliding_window_variance(column, window_size)

# Pad the variance_values array with NaNs to match the length of the DataFrame's index
padding = np.empty(len(df.index) - len(variance_values))
padding[:] = np.nan
variance_values = np.concatenate((variance_values, padding))

# Save the result to a new column in the DataFrame
df["Ecell_Variance"] = variance_values

# -------------------------------------------------------------------

# Select the column of interest
column = df["I_mA"]

window_size = 10
variance_values = sliding_window_variance(column, window_size)

# Pad the variance_values array with NaNs to match the length of the DataFrame's index
padding = np.empty(len(df.index) - len(variance_values))
padding[:] = np.nan
variance_values = np.concatenate((variance_values, padding))

# Save the result to a new column in the DataFrame
df["I_mA_Variance"] = variance_values

# ---------------------------------------------------------------------

# Select the column of interest
column = df["Temperature__C"]

window_size = 10
variance_values = sliding_window_variance(column, window_size)

# Pad the variance_values array with NaNs to match the length of the DataFrame's index
padding = np.empty(len(df.index) - len(variance_values))
padding[:] = np.nan
variance_values = np.concatenate((variance_values, padding))

# Save the result to a new column in the DataFrame
df["Temperature__C_Variance"] = variance_values

In [10]:
# --------------------------------------------------------Power--------------------------------------------------------

# Select the columns of interest
voltage_column = df["Ecell_V"]
current_column = df["I_mA"]

# Multiply the columns to get the power
power_column = (voltage_column * current_column)/1000

# Save the result to a new column in the DataFrame
df["Power"] = power_column
df


# ---------------------------------------------------------Resistance--------------------------------------------------


# Select the columns of interest
voltage_column = df["Ecell_V"]
current_column = df["I_mA"]

# Multiply the columns to get the power
resistance_column = (voltage_column / current_column)*1000

# Save the result to a new column in the DataFrame
df["Resistance"] = resistance_column
df


# ----------------------------------------------------------Conductance------------------------------------------------




conductance_column = 1/resistance_column

# Save the result to a new column in the DataFrame
df["Conductance"] = conductance_column
df




# ---------------------------------------------------------Temperature Differnce---------------------------------------------
# Select the column of interest
temp_column = df["Temperature__C"]

# Calculate the difference between consecutive rows
temp_change = temp_column.diff()
# Fill the first row with 0
temp_change.iloc[0] = 0


# Save the result to a new column in the DataFrame
df["temp_change"] = temp_change
df

,Ecell_V,I_mA,QDischarge_mA_h,Temperature__C,SOC%,EcellMean,I_Mean,TemperatureMean,EcellMedian,ImaMedian,...,Ecell_Vstd,I_mA_std,TempStd,Ecell_Variance,I_mA_Variance,Temperature__C_Variance,Power,Resistance,Conductance,temp_change
0,3.538717,1491.3828,0.000000,21.139246,100.000000,3.554757,1491.99352,21.653207,3.556596,1491.67830,...,0.007640,0.891832,0.369085,0.000058,0.795365,0.136224,5.277582,2.372776,0.421447,0.000000
1,3.545570,1492.1708,0.000000,21.115524,100.000000,3.557431,1492.07233,21.777349,3.558487,1491.97385,...,0.006078,0.868909,0.383817,0.000037,0.755003,0.147316,5.290596,2.376115,0.420855,-0.023722
2,3.549980,1491.5797,0.000000,21.305294,100.000000,3.559566,1492.01323,21.896746,3.560180,1491.67835,...,0.005226,0.880226,0.342895,0.000027,0.774799,0.117577,5.295079,2.380014,0.420166,0.189770
3,3.553131,1492.3679,0.000000,21.574135,100.000000,3.561413,1491.91473,22.016934,3.561775,1491.67835,...,0.004754,0.973409,0.324673,0.000023,0.947524,0.105413,5.302578,2.380868,0.420015,0.268841
4,3.555572,1490.9886,0.000000,21.510880,100.000000,3.563087,1491.79652,22.117354,3.563311,1491.57975,...,0.004482,0.982928,0.327473,0.000020,0.966148,0.107239,5.301318,2.384708,0.419339,-0.063255
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,3.591331,-4464.8276,1088.086524,33.996197,63.730449,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-16.034673,-0.804360,-1.243224,0.150238
1048571,3.586133,-4471.3296,1100.499700,33.774796,63.316677,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-16.034780,-0.802028,-1.246839,-0.221401
1048572,3.580974,-4477.8306,1112.930732,33.988289,62.902309,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-16.034993,-0.799712,-1.250451,0.213493
1048573,3.575775,-4484.3325,1125.379697,33.964565,62.487343,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,-16.034964,-0.797393,-1.254087,-0.023724


In [11]:
# ------------------------------------------------------------Data Cleaning----------------------------------------

# Replace inf values with NaN
df.replace([np.inf, -np.inf], np.nan, inplace=True)



# Remove rows containing NaN values
df.dropna(inplace=True)


# Initialize the scaler
scaler = MinMaxScaler()

# Fit and transform the data
df_scaled = scaler.fit_transform(df)

# Put the transformed data back into a dataframe
df = pd.DataFrame(df_scaled, columns=df.columns)


index1=0
index2=20998
index3=20729
index4=24457
index5=26176
index6=27894
df

,Ecell_V,I_mA,QDischarge_mA_h,Temperature__C,SOC%,EcellMean,I_Mean,TemperatureMean,EcellMedian,ImaMedian,...,Ecell_Vstd,I_mA_std,TempStd,Ecell_Variance,I_mA_Variance,Temperature__C_Variance,Power,Resistance,Conductance,temp_change
0,0.610938,0.937774,0.000000,0.022402,1.000000,0.619538,0.935577,0.033404,0.620635,0.936120,...,0.017250,0.000071,0.035752,0.000298,7.160660e-09,0.001378,0.890580,0.395941,0.943932,0.367971
1,0.614968,0.937807,0.000000,0.021885,1.000000,0.621114,0.935580,0.036532,0.621750,0.936133,...,0.013717,0.000069,0.037237,0.000189,6.786128e-09,0.001490,0.890775,0.395962,0.943868,0.364303
2,0.617562,0.937782,0.000000,0.026021,1.000000,0.622373,0.935577,0.039539,0.622748,0.936120,...,0.011790,0.000070,0.033113,0.000140,6.969820e-09,0.001189,0.890842,0.395987,0.943794,0.397311
3,0.619415,0.937815,0.000000,0.031880,1.000000,0.623462,0.935573,0.042567,0.623688,0.936120,...,0.010724,0.000079,0.031276,0.000116,8.572572e-09,0.001066,0.890955,0.395992,0.943777,0.409536
4,0.620851,0.937758,0.000000,0.030501,1.000000,0.624448,0.935568,0.045097,0.624594,0.936116,...,0.010109,0.000080,0.031558,0.000103,8.745388e-09,0.001084,0.890936,0.396016,0.943704,0.358191
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763319,0.667825,0.693637,0.316294,0.307255,0.683706,0.654295,0.680883,0.345776,0.654451,0.683700,...,0.031865,0.001676,0.015002,0.001017,2.857063e-06,0.000268,0.570821,0.375808,0.767019,0.350856
763320,0.665115,0.693401,0.320263,0.309667,0.679737,0.651388,0.680624,0.344681,0.651525,0.683385,...,0.032228,0.001688,0.016021,0.001040,2.898937e-06,0.000302,0.570815,0.375822,0.766682,0.385086
763321,0.662266,0.693189,0.324237,0.301913,0.675763,0.648449,0.680362,0.343844,0.648554,0.683113,...,0.032531,0.001702,0.012564,0.001060,2.945815e-06,0.000194,0.570857,0.375835,0.766353,0.312959
763322,0.659510,0.692871,0.328217,0.304842,0.671783,0.645489,0.680093,0.343844,0.645582,0.682840,...,0.032848,0.001703,0.012564,0.001081,2.951536e-06,0.000194,0.570751,0.375850,0.765953,0.388754


In [12]:
df_training = df.iloc[index5+1:index6]
features = ['Ecell_V','I_mA', 'Temperature__C','EcellMean','I_Mean','TemperatureMean','EcellMedian','ImaMedian','TempMedian','Ecell_Vstd','I_mA_std','TempStd','Ecell_Variance','I_mA_Variance','Temperature__C_Variance','Power','Resistance','Conductance','temp_change']
X = df_training.loc[:, features]
y = df_training.loc[:, ['SOC%']]
df_training.shape

(1717, 21)

In [13]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1)


In [14]:
# Table
dfAnswer = pd.DataFrame(columns=['Algo', 'RMSE', 'MAE', 'R-Squared'])

In [16]:
# 1.___________________________________OLS______________________________________________________________

ols = LinearRegression()
ols.fit(X_train, y_train)
ols_yhat_test = ols.predict(X_test)

# Predict the output for the test set
y_pred = ols.predict(X_test)

# Rmse
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Calculate the Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)

# Calculate the R2 score
r2 = r2_score(y_test, y_pred)


dfAnswer = dfAnswer.append({'Algo': 'ols', 'RMSE': rmse, 'MAE':mae, 'R-Squared':r2}, ignore_index=True)

print(dfAnswer)

  Algo      RMSE       MAE  R-Squared
0  ols  0.021843  0.013982   0.976602


In [19]:
#2 ________________________________Decision Tree______________________________________________________________________________


# Create decision tree regressor
decReg = DecisionTreeRegressor()

# Fit regressor to training data
decReg.fit(X_train, y_train)

# Predict the output for the test set
y_pred = decReg.predict(X_test)

# Rmse
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Calculate the Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)

# Calculate the R2 score
r2 = r2_score(y_test, y_pred)

dfAnswer = dfAnswer.append({'Algo': 'DecisionTree', 'RMSE': rmse, 'MAE':mae, 'R-Squared':r2}, ignore_index=True)

print(dfAnswer)

           Algo      RMSE       MAE  R-Squared
0           ols  0.021843  0.013982   0.976602
1  DecisionTree  0.003039  0.000972   0.999547


In [20]:
# 3------------------------------------------------RandomForest---------------------------------------------------------------

# create regressor object
randomForestReg = RandomForestRegressor()
 
# fit the regressor with x and y data
randomForestReg.fit(X_train, y_train) 

# Predict the output for the test set
y_pred = randomForestReg.predict(X_test)

# Rmse
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Calculate the Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)

# Calculate the R2 score
r2 = r2_score(y_test, y_pred)

dfAnswer = dfAnswer.append({'Algo': 'RandomForest', 'RMSE': rmse, 'MAE':mae, 'R-Squared':r2}, ignore_index=True)

print(dfAnswer)

<ipython-input-20-5de7c63d3f1c>:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  randomForestReg.fit(X_train, y_train)


           Algo      RMSE       MAE  R-Squared
0           ols  0.021843  0.013982   0.976602
1  DecisionTree  0.003039  0.000972   0.999547
2  RandomForest  0.002909  0.000750   0.999585


In [21]:
# ___________________________________________________________GradientBoostingRegression--------------------------------------

# Instantiate Gradient Boosting Regressor
gbr = GradientBoostingRegressor(n_estimators = 32, max_depth = 1, random_state = 1)
  
# Fit to training set
gbr.fit(X_train, y_train)
  
# Predict the output for the test set
y_pred = gbr.predict(X_test)

# Rmse
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Calculate the Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)

# Calculate the R2 score
r2 = r2_score(y_test, y_pred)

dfAnswer = dfAnswer.append({'Algo': 'GradientBoosting', 'RMSE': rmse, 'MAE':mae, 'R-Squared':r2}, ignore_index=True)

print(dfAnswer)

               Algo      RMSE       MAE  R-Squared
0               ols  0.021843  0.013982   0.976602
1      DecisionTree  0.003039  0.000972   0.999547
2      RandomForest  0.002909  0.000750   0.999585
3  GradientBoosting  0.052698  0.034869   0.863819


/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [22]:
# ___________________________________XTREME GRADIENT BOOSTING_____________________________________________________________

# Create XGBoost regressor
xgbr = xgb.XGBRegressor()

# Fit regressor to training data
xgbr.fit(X_train, y_train)

# Predict the output for the test set
y_pred = xgbr.predict(X_test)

# Rmse
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

# Calculate the Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)

# Calculate the R2 score
r2 = r2_score(y_test, y_pred)

dfAnswer = dfAnswer.append({'Algo': 'XtremeGradientBoost', 'RMSE': rmse, 'MAE':mae, 'R-Squared':r2}, ignore_index=True)

print(dfAnswer)

[08:58:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
                  Algo      RMSE       MAE  R-Squared
0                  ols  0.021843  0.013982   0.976602
1         DecisionTree  0.003039  0.000972   0.999547
2         RandomForest  0.002909  0.000750   0.999585
3     GradientBoosting  0.052698  0.034869   0.863819
4  XtremeGradientBoost  0.004833  0.002054   0.998855


In [23]:
# -----------------------------------------------Neural Network----------------------------------------------------------


# Define the Shallow Neural Network (NN)
nn_model = Sequential()
nn_model.add(Dense(500, activation='sigmoid', input_shape=(19,)))
nn_model.add(Dense(1, activation='linear'))
nn_model.compile(optimizer='adam', loss='mean_squared_error')

# Train the NN
nn_history = nn_model.fit(X_train, y_train, epochs=50, batch_size=16, verbose=0)

 # Predict the output for the test set
y_pred = nn_model.predict(X_test)

# Calculate the Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)

# Calculate the Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)

# Calculate the R2 score
r2 = r2_score(y_test, y_pred)

dfAnswer = dfAnswer.append({'Algo': 'NN', 'RMSE': rmse, 'MAE':mae, 'R-Squared':r2}, ignore_index=True)

print(dfAnswer)

22/22 [==============================] - 0s 2ms/step
                  Algo      RMSE       MAE  R-Squared
0                  ols  0.021843  0.013982   0.976602
1         DecisionTree  0.003039  0.000972   0.999547
2         RandomForest  0.002909  0.000750   0.999585
3     GradientBoosting  0.052698  0.034869   0.863819
4  XtremeGradientBoost  0.004833  0.002054   0.998855
5                   NN  0.004833  0.020747   0.928931


In [24]:
# ----------------------------------------------DEEP NEURAL NETWORK----------------------------------------------------------

# Define the Deep Neural Network (DNN)
dnn_model = Sequential()
dnn_model.add(Dense(500, activation='sigmoid', input_shape=(19,)))
dnn_model.add(Dense(500, activation='sigmoid'))
dnn_model.add(Dense(500, activation='sigmoid'))
dnn_model.add(Dense(500, activation='sigmoid'))
dnn_model.add(Dense(1, activation='linear'))
dnn_model.compile(optimizer='adam', loss='mean_squared_error')

# Train the DNN
dnn_history = dnn_model.fit(X_train, y_train, epochs=50, batch_size=16, verbose=0)

# Predict the output for the test set
y_pred = dnn_model.predict(X_test)

# Calculate the Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)

# Calculate the Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)

# Calculate the R2 score
r2 = r2_score(y_test, y_pred)

dfAnswer = dfAnswer.append({'Algo': 'DNN', 'RMSE': rmse, 'MAE':mae, 'R-Squared':r2}, ignore_index=True)

print(dfAnswer)

22/22 [==============================] - 0s 3ms/step
                  Algo      RMSE       MAE  R-Squared
0                  ols  0.021843  0.013982   0.976602
1         DecisionTree  0.003039  0.000972   0.999547
2         RandomForest  0.002909  0.000750   0.999585
3     GradientBoosting  0.052698  0.034869   0.863819
4  XtremeGradientBoost  0.004833  0.002054   0.998855
5                   NN  0.004833  0.020747   0.928931
6                  DNN  0.004833  0.044123   0.830410


In [25]:
# _____________________________________________________SVM USING LINEAR KERNEL------------------------------------


linear_svr = SVR(kernel='linear', cache_size=2097152)

# Fit the model to the training data
linear_svr.fit(X_train, y_train)

y_pred = linear_svr.predict(X_test)

# Calculate the Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)

# Calculate the Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)

# Calculate the R2 score
r2 = r2_score(y_test, y_pred)

dfAnswer = dfAnswer.append({'Algo': 'SVM Linear', 'RMSE': rmse, 'MAE':mae, 'R-Squared':r2}, ignore_index=True)

print(dfAnswer)

                  Algo      RMSE       MAE  R-Squared
0                  ols  0.021843  0.013982   0.976602
1         DecisionTree  0.003039  0.000972   0.999547
2         RandomForest  0.002909  0.000750   0.999585
3     GradientBoosting  0.052698  0.034869   0.863819
4  XtremeGradientBoost  0.004833  0.002054   0.998855
5                   NN  0.004833  0.020747   0.928931
6                  DNN  0.004833  0.044123   0.830410
7           SVM Linear  0.004833  0.072772   0.710399


/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [26]:
# Create a support vector machine with a radial-basis function (RBF) kernel
rbf_svr = SVR(kernel='rbf', cache_size=2097152)

# Fit the model to the training data
rbf_svr.fit(X_train, y_train)

y_pred = rbf_svr.predict(X_test)

# Calculate the Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)

# Calculate the Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)

# Calculate the R2 score
r2 = r2_score(y_test, y_pred)

dfAnswer = dfAnswer.append({'Algo': 'SVM RBF', 'RMSE': rmse, 'MAE':mae, 'R-Squared':r2}, ignore_index=True)

print(dfAnswer)

                  Algo      RMSE       MAE  R-Squared
0                  ols  0.021843  0.013982   0.976602
1         DecisionTree  0.003039  0.000972   0.999547
2         RandomForest  0.002909  0.000750   0.999585
3     GradientBoosting  0.052698  0.034869   0.863819
4  XtremeGradientBoost  0.004833  0.002054   0.998855
5                   NN  0.004833  0.020747   0.928931
6                  DNN  0.004833  0.044123   0.830410
7           SVM Linear  0.004833  0.072772   0.710399
8              SVM RBF  0.004833  0.052211   0.827790


/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
